# Connection to Library DB

In [20]:

import mysql.connector

library = mysql.connector.connect(user='root', password='1514',
                              host='127.0.0.1',
                              database='library')

In [21]:
cursor = library.cursor()

In [22]:
# Test query 
query = ("SELECT * FROM library.books;")

cursor.execute(query)

data = cursor.fetchmany(5)
for x in data:
    print(x)

(1, 'Walden, and On The Duty Of Civil Disobedience', 'Henry David Thoreau', 1849, 3100471153)
(2, 'The Scarlet Letter', 'Nathaniel Hawthorne', 1850, 4589408958)
(3, 'The Adventures of Sherlock Holmes', 'Arthur Conan Doyle', 1892, 2411676280)
(4, 'The Adventures of Tom Sawyer', 'Mark Twain', 1876, 7813216705)
(5, 'David Copperfield', 'Charles Dickens', 1850, 8821265677)


# Find the number of availabe copies of Dracula

In [24]:
# check total copies of the book
library.reconnect()
query = ('''    SELECT  COUNT(BookID) AS TotalCopies
                FROM Books
                WHERE Title LIKE '%Dracula%';            ''')

cursor.execute(query)

data = cursor.fetchall()
for x in data:
    print(x)

(4,)


In [25]:
# current total loans of the book

query = ('''    SELECT *, COUNT(LoanID) AS TotalLoans
                FROM Loans
                WHERE BookID IN (
                SELECT BookID FROM Books WHERE Title LIKE '%Dracula%')
                AND ReturnedDate IS NULL;            ''')

cursor.execute(query)

data = cursor.fetchall()
for x in data:
    print(x)

(1999, 73, 59, '2020-06-29', '2020-07-13', None, 1)


In [26]:
# total available book

query = ('''    SELECT
	                    (SELECT COUNT(BookID) AS TotalCopies
	                    FROM Books
	                    WHERE Title LIKE '%Dracula%')
	            -
	                    (SELECT COUNT(LoanID) AS TotalLoans
	                    FROM Loans
	                    WHERE BookID IN (SELECT BookID FROM Books WHERE Title LIKE '%Dracula%')
	                    AND ReturnedDate IS NULL) AS AvailableBooks;            ''')

cursor.execute(query)

data = cursor.fetchall()
for x in data:
    print(x)

(3,)


# Check books for Due back                             
> generate a report of books due back on July 13, 2020 with patron contact information

In [7]:
query = ('''    SELECT p.FirstName, p.LastName, p.Email, b.Title, l.LoanDate, l.DueDate
                FROM Loans l
                JOIN Books b ON l.BookID = b.BookID
                JOIN Patrons p ON l.PatronID = p.PatronID
                WHERE l.DueDate = '2020-07-13'
                AND ReturnedDate IS NULL;          ''')

cursor.execute(query)

data = cursor.fetchall()
for x in data:
    print(x)

('Kala', 'Simonian', 'ksimonian2q@landonhotel.com', 'The Count of Monte Cristo', '2020-06-29', '2020-07-13')
('Lydie', 'Awmack', 'lawmack1a@samoca.org', 'The Prophet', '2020-06-29', '2020-07-13')
('Anthe', 'Dinjes', 'adinjesh@samoca.org', 'Second Treatise of Government', '2020-06-29', '2020-07-13')
('Cornelia', 'Koppe', 'ckoppe1m@landonhotel.com', 'Dracula', '2020-06-29', '2020-07-13')


# Return books to the library

In [8]:
query = ('''    SELECT * FROM Loans
                WHERE BookID IN (SELECT BookID FROM Books
                WHERE Barcode = 6435968624)
                AND ReturnedDate IS NULL;

                UPDATE Loans
                SET ReturnedDate = '2020-07-05'
                WHERE BookID IN 
                    (SELECT BookID FROM Books WHERE Barcode = 6435968624)
                AND ReturnedDate IS NULL;         ''')

cursor.execute(query)

data = cursor.fetchall()
for x in data:
    print(x)

# Encourage Patrons to check out books
> generate a report of showing 10 patrons who have checked out the fewest books.

In [9]:
library.reconnect()
cursor = library.cursor()
query = ('''    SELECT p.FirstName, p.LastName, p.Email, COUNT(p.PatronID) AS Total_Loans
                FROM Patrons p
                LEFT JOIN Loans l
                ON p.PatronID = l.PatronID
                GROUP BY p.PatronID
                ORDER BY 4 ASC
                LIMIT 10;       ''')

cursor.execute(query)

data = cursor.fetchall()
for x in data:
    print(x)        

('Caril', 'Matejic', 'cmatejic2a@orangevalleycaa.org', 9)
('Alaster', 'Ruggles', 'aruggles1w@red30design.com', 9)
('Merle', 'Sukbhans', 'msukbhans20@orangevalleycaa.org', 11)
('Norby', 'Sleight', 'nsleight1r@kinetecoinc.com', 12)
('Curr', 'Fittall', 'cfittall2l@orangevalleycaa.org', 12)
('Cammi', 'Kynett', 'ckynett6@orangevalleycaa.org', 14)
('Aleen', 'Fasey', 'afasey2@kinetecoinc.com', 14)
('Orton', 'Stavers', 'ostaversb@orangevalleycaa.org', 14)
('Carla', 'Swinfen', 'cswinfenf@kinetecoinc.com', 14)
('Don', 'Weingarten', 'dweingarten@kinetecoinc.com', 15)


# Find books to feature for an event                  
 > create a list of books from 1890s that are currently available   

In [10]:
query = ('''    SELECT b.BookID, b.Title, b.Author, b.Published, COUNT(b.BookID) AS TotalAvailableBooks
                FROM Books b
                LEFT JOIN Loans l
                ON b.BookID = l.BookID
                WHERE ReturnedDate IS NOT NULL
                AND b.Published BETWEEN 1890 AND 1899
                GROUP BY b.BookID
                ORDER BY b.BookID;       ''')

cursor.execute(query)

data = cursor.fetchall()
for x in data:
    print(x)        

(3, 'The Adventures of Sherlock Holmes', 'Arthur Conan Doyle', 1892, 5)
(7, 'The Sign of the Four', 'Arthur Conan Doyle', 1890, 13)
(8, 'The Importance of Being Earnest: A Trivial Comedy for Serious People', 'Oscar Wilde', 1895, 10)
(11, 'The Picture of Dorian Gray', 'Oscar Wilde', 1890, 15)
(12, 'Dracula', 'Bram Stoker', 1897, 6)
(14, 'The Awakening, and Selected Short Stories', 'Kate Chopin', 1899, 16)
(15, 'The Time Machine', 'H. G. Wells', 1895, 9)
(17, 'Les Misérables', 'Victor Hugo', 1895, 5)
(18, 'The Jungle Book', 'Rudyard Kipling', 1894, 7)
(19, 'Heart of Darkness', 'Joseph Conrad', 1899, 13)
(26, 'The Adventures of Sherlock Holmes', 'Arthur Conan Doyle', 1892, 14)
(28, 'Heart of Darkness', 'Joseph Conrad', 1899, 6)
(43, 'The Yellow Wallpaper', 'Charlotte Perkins Gilman', 1892, 12)
(54, 'The Yellow Wallpaper', 'Charlotte Perkins Gilman', 1892, 11)
(55, 'An Occurrence at Owl Creek Bridge', 'Ambrose Bierce', 1890, 14)
(56, 'The Picture of Dorian Gray', 'Oscar Wilde', 1890, 10)
(

# Book Statistics 


## create a report to show how many books were published each year.  

In [11]:
query = ('''    SELECT Published, COUNT(DISTINCT(Title)) AS TotalNumberOfPublishedBooks
                FROM Books
                GROUP BY Published
                ORDER BY TotalNumberOfPublishedBooks DESC;       ''')

cursor.execute(query)

data = cursor.fetchall()
for x in data:
    print(x)        

(1895, 3)
(1890, 3)
(1847, 2)
(1911, 2)
(1884, 2)
(1859, 2)
(1898, 2)
(1899, 2)
(1729, 2)
(1892, 2)
(1850, 2)
(1869, 2)
(1922, 2)
(1532, 1)
(1605, 1)
(1670, 1)
(1689, 1)
(1726, 1)
(1791, 1)
(1811, 1)
(1813, 1)
(1815, 1)
(1818, 1)
(1839, 1)
(1759, 1)
(1849, 1)
(1851, 1)
(1852, 1)
(1854, 1)
(1858, 1)
(1861, 1)
(1865, 1)
(1876, 1)
(1889, 1)
(1879, 1)
(1880, 1)
(1894, 1)
(1878, 1)
(1896, 1)
(1897, 1)
(1883, 1)
(1886, 1)
(1900, 1)
(1901, 1)
(1902, 1)
(1905, 1)
(1906, 1)
(1908, 1)
(1375, 1)
(1913, 1)
(1914, 1)
(1915, 1)
(1920, 1)
(1921, 1)
(1887, 1)
(1923, 1)


## create a report to show 5 most popular Books to check out

In [12]:
query = ('''    SELECT b.Title, b.Author, b.Published, COUNT(b.Title) AS TotalTimesOfLoans
                FROM Books b
                JOIN Loans l
                ON b.BookID = l.BookID
                GROUP BY b.Title
                ORDER BY 4 DESC
                LIMIT 5;      ''')

cursor.execute(query)

data = cursor.fetchall()
for x in data:
    print(x)        

('Frankenstein; Or, The Modern Prometheus', 'Mary Wollstonecraft Shelley', 1818, 53)
('The Picture of Dorian Gray', 'Oscar Wilde', 1890, 44)
('The Awakening, and Selected Short Stories', 'Kate Chopin', 1899, 43)
('The War of the Worlds', 'H. G. Wells', 1898, 43)
('The Scarlet Letter', 'Nathaniel Hawthorne', 1850, 42)


# Add new books to the library

In [13]:
query = ('''    INSERT IGNORE INTO Books(Title, Author, Published, Barcode)
                VALUES
                ('Dracula', 'Bram Stoker', 1897, 4819277482),
                ('Gulliver''s Travel', 'Johnathan Swift',1729,4899254401);          ''')

cursor.execute(query)

data = cursor.fetchall()
for x in data:
    print(x)

In [14]:
library.commit()

# Check out Books

In [15]:

query = ('''    INSERT INTO Loans(BookID, PatronID, LoanDate, DueDate)
            VALUES
            (
                (SELECT BookID FROM Books WHERE Barcode = 4043822646),
                (SELECT PatronID FROM Patrons WHERE Email LIKE 'jvaan@wisdompets.com'),
                '2020-08-25',
                '2020-09-08'
            ),
            (
                (SELECT BookID FROM Books WHERE Barcode = 2855934983),
                (SELECT PatronID FROM Patrons WHERE Email LIKE 'jvaan@wisdompets.com'),
                '2020-08-25',
                '2020-09-08'
            );

            SELECT * FROM Loans
            ORDER BY LoanID DESC
            LIMIT 5;            ''')

cursor.execute(query)

data = cursor.fetchall()
for x in data:
    print(x)

In [17]:
library.reconnect()

query = ('''    SELECT b.Title, b.Author, b.Published, COUNT(b.Title) AS TotalTimesOfLoans
                FROM Books b
                JOIN Loans l
                ON b.BookID = l.BookID
                GROUP BY b.Title
                ORDER BY 4 DESC
                LIMIT 5;            ''')

cursor.execute(query)

data = cursor.fetchall()
for x in data:
    print(x)

('Frankenstein; Or, The Modern Prometheus', 'Mary Wollstonecraft Shelley', 1818, 53)
('The Picture of Dorian Gray', 'Oscar Wilde', 1890, 44)
('The Awakening, and Selected Short Stories', 'Kate Chopin', 1899, 43)
('The War of the Worlds', 'H. G. Wells', 1898, 43)
('The Scarlet Letter', 'Nathaniel Hawthorne', 1850, 42)
